In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Loading data
data = pd.read_csv('INFY20002008.csv')
data.info()

In [ ]:
# Selecting three columns as input 
data = data[['Last Price', 'Average Price', 'Open Price']].values

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(data[:,0]+500, '--g', label='Last Price')
plt.plot(data[:,1], '--r', label='Average Price')
plt.plot(data[:,2]-500, '--b', label='Open Price')
plt.xlabel('Days', fontsize=15)
plt.ylabel('Price', fontsize=15)
plt.suptitle('Plot of Last, Average and Open price', fontsize=18, weight='bold')
plt.title('with added noise to prevent overlapping')
plt.legend()
plt.show()

In [ ]:
# Scaling the values in the range of 0 to 1
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data[:,:])

In [ ]:
# Splitting dataset in the ratio of 75:25 for training and test
train_size = int(scaled_data.shape[0] * 0.75)
train, test = scaled_data[0:train_size, :], scaled_data[train_size:scaled_data.shape[0], :]
print("Number of entries (training set, test set): " + str((len(train), len(test))))

In [ ]:
def create_dataset(ser, window_size=1):
    data_X, data_Y = [], []
    for i in range(len(ser) - window_size - 1):
        a = []
        for j in range(0, window_size):        
            a.append(ser[[(i+j)], :])
        data_X.append(a)
        data_Y.append(ser[i + window_size, 1])
    return(np.array(data_X), np.array(data_Y))

In [ ]:
window_size = 50
train_X, train_Y = create_dataset(train, window_size)
test_X, test_Y = create_dataset(test, window_size)
print("Original training data shape:")
print(train_X.shape, test_X.shape)

# Reshape the input data into appropriate form for Keras.
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 3))
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], 3))
print("New training data shape:")
print(train_X.shape, test_X.shape)

In [ ]:
# Designing the LSTM model
model = Sequential()
model.add(LSTM(4, input_shape = (window_size, 3)))
model.add(Dense(1))

# Compiling the model
model.compile(loss = "mean_squared_error", optimizer = "adam")

# Training the model
model.fit(train_X, train_Y, epochs=3, batch_size=1)

In [ ]:
# Visualization
plt.figure(figsize = (15, 5))
plt.plot(np.concatenate((train[:,1], test[:,1])), '--b', label='True value')

plt.plot(range(window_size, len(train_X) + window_size), model.predict(train_X), '-r', label='Training set prediction')
plt.plot(range(window_size + len(train), len(train) + len(test) - 1), model.predict(test_X), '-g', label='Test set prediction')

plt.xlabel("Days")
plt.ylabel("Average Price")
plt.title("Comparison true vs. predicted training / test", fontsize=16)
plt.legend()
plt.xlim(-5, 2500)
plt.show()